<a href="https://colab.research.google.com/github/DeepakGoli/Multinomial-Naive-Bayes-/blob/master/MultinomialNaive_NewsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import operator
nltk.download('stopwords')
from google.colab import drive
drive.mount('/content/drive/')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [86]:
#calculating prior probabilites
path_to_dataset = '/content/drive/My Drive/statisticalPR/datasets/NewsDataSet/'
train_label = open(path_to_dataset+'train.label')
iter_label = train_label.readlines()
prior_probs = {}
for i in range(1,21):
    prior_probs[i]=0
for i in iter_label:
    prior_probs[int(i)]=prior_probs[int(i)]+1
for key in prior_probs:
    prior_probs[key]=prior_probs[key]/len(iter_label)
    
print("Probability of each class:")
print("\n".join("{}: {}".format(k, v) for k, v in prior_probs.items()))

Probability of each class:
1: 0.04259472890229834
2: 0.05155736977549028
3: 0.05075871860857219
4: 0.05208980388676901
5: 0.051024935664211554
6: 0.052533498979501284
7: 0.051646108794036735
8: 0.052533498979501284
9: 0.052888455053687104
10: 0.0527109770165942
11: 0.05306593309078002
12: 0.0527109770165942
13: 0.05244475996095483
14: 0.0527109770165942
15: 0.052622237998047744
16: 0.05315467210932647
17: 0.04836276510781791
18: 0.05004880646020055
19: 0.04117490460555506
20: 0.033365870973467035


In [0]:
#finding the relevant vocabulary
stop_words = set(stopwords.words('english'))
vocab = open(path_to_dataset+'vocabulary.txt') 
iter_vocab = vocab.readlines()
dict_vocab = {}
for idx, i in enumerate(iter_vocab):
    dict_vocab[idx+1]=i

dict_good_vocab = {}
#good vocabulary has the key 1 
for key in dict_vocab:
    if dict_vocab[key][:-1] in stop_words: dict_good_vocab[key]=0
    else : dict_good_vocab[key]=1

#finding the most relevant vocabulary. Top n words from the dataset
train_data = open(path_to_dataset+'train.data')
iter_train_data = train_data.readlines()
vocab_freq={}
for key in dict_vocab:
    vocab_freq[key]=0

for i in iter_train_data:
    data = i.split()
    vocab_freq[int(data[1])]=vocab_freq[int(data[1])]+int(data[2])
sorted_vocab_freq = sorted(vocab_freq.items(), key=operator.itemgetter(1),reverse=True)
#All the ID's of top n words that are not from the stop words are stored in relevant_vocab array
relevant_vocab=[]
n=500
for i in range(n):
    key = sorted_vocab_freq[i][0]
    if dict_good_vocab[key]==1:
        relevant_vocab.append(key)



In [167]:
#creating a pandas dataframe to visualize data

train_data = open(path_to_dataset+'train.data')
df = pd.read_csv(train_data,delimiter=' ',names=['docIdx','wordIdx','count'])
label=[]
for i in iter_label:
    label.append(int(i.split()[0]))
docIdx = df['docIdx'].values
i=0
new_label = []
for index in range(len(docIdx)-1):
    new_label.append(label[i])
    if docIdx[index] != docIdx[index+1]:
        i=i+1
new_label.append(label[i])
df['classIdx'] = new_label
print(df.head())

#Use the relevant_vocab array to 
df=df[df['wordIdx'].isin(relevant_vocab)]
print(df.head())
print(len(relevant_vocab))

   docIdx  wordIdx  count  classIdx
0       1        1      4         1
1       1        2      2         1
2       1        3     10         1
3       1        4      4         1
4       1        5      2         1
    docIdx  wordIdx  count  classIdx
1        1        2      2         1
5        1        6      1         1
8        1        9      3         1
16       1       17      6         1
27       1       28      2         1
385


In [163]:
#Calculate probability of each word based on class
pb_ij = df.groupby(['classIdx','wordIdx'])
pb_j = df.groupby(['classIdx'])
#laplace smoothing
a=0.001
Pr = (pb_ij['count'].sum() + a) / (pb_j['count'].sum()+a*len(relevant_vocab))    

#Unstack series
Pr = Pr.unstack()

#Replace NaN or columns with 0 as word count with a/(count+|V|+1)
for c in range(1,21):
    Pr.loc[c,:] = Pr.loc[c,:].fillna(a/(pb_j['count'].sum()[c]))

#Convert to dictionary for greater speed
Pr_dict = Pr.to_dict()
Pr

wordIdx,1,2,3,4,5,6,7,8,9,10,11,13,14,15,17,18,19,20,21,22,24,26,28,31,32,34,35,36,37,38,39,40,41,43,44,45,46,47,50,53,...,53852,53855,53856,53857,53858,53859,53860,53861,53862,53863,53872,53877,53879,53880,53881,53886,53899,53904,53910,53919,53921,53927,53929,53930,53932,53933,53934,53937,53938,53939,53945,53948,53949,53956,53957,53958,53961,53963,53965,53968
classIdx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.652645e-04,0.000801,3.495723e-03,1.144178e-04,0.001042,0.000521,7.628275e-05,1.272438e-05,0.000432,1.779647e-03,3.814773e-05,2.033995e-04,0.000229,5.211913e-04,2.809292e-03,1.271294e-04,8.899442e-05,1.398411e-04,1.272438e-05,1.272438e-05,3.814773e-05,1.272438e-05,0.000203,0.002542,0.000292,1.272438e-05,0.000153,2.543606e-05,3.814773e-05,5.085940e-05,3.686512e-04,3.814773e-05,0.000038,3.814773e-05,0.003559,0.000445,7.754247e-04,0.000127,7.628275e-05,5.085940e-05,...,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08,1.271973e-08
2,8.628413e-04,0.000848,1.439077e-08,2.444820e-04,0.000201,0.000834,1.438189e-04,8.629708e-05,0.002473,4.315573e-05,2.157211e-04,1.438189e-04,0.000273,1.439483e-05,1.439077e-08,4.315573e-05,1.439077e-08,5.753618e-05,2.877528e-05,1.439077e-08,1.439483e-05,1.439077e-08,0.002847,0.000575,0.001150,1.439077e-08,0.000561,1.439077e-08,1.439077e-08,1.438189e-04,1.439483e-05,1.439483e-05,0.000086,1.439077e-08,0.003264,0.000144,1.439077e-08,0.000043,2.877528e-05,1.439483e-05,...,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08,1.439077e-08
3,1.933567e-04,0.001213,1.759170e-08,2.988144e-04,0.000369,0.000598,3.517014e-05,3.517014e-05,0.002531,1.759170e-08,1.054753e-04,1.759386e-05,0.000229,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759386e-05,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,0.001459,0.000844,0.000685,1.759170e-08,0.000615,1.759170e-08,1.582041e-04,7.032270e-05,1.759170e-08,5.274642e-05,0.000105,3.517014e-05,0.004130,0.000158,1.759170e-08,0.000053,1.759170e-08,5.274642e-05,...,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08,1.759170e-08
4,1.335150e-04,0.000517,1.670118e-08,1.670118e-08,0.000167,0.000801,3.339126e-05,1.670397e-05,0.000801,1.670118e-08,1.502023e-04,1.670118e-08,0.000317,1.670397e-05,1.670118e-08,1.670118e-08,3.339126e-05,6.676584e-05,1.670118e-08,1.670118e-08,1.670118e-08,1.670118e-08,0.001018,0.000751,0.000768,1.670118e-08,0.000501,1.670118e-08,1.670397e-05,8.345313e-05,1.670397e-05,6.676584e-05,0.000184,1.670118e-08,0.003988,0.000384,1.670118e-08,0.000067,1.670118e-08,1.670118e-08,...,1.670118e-08,1.670118e-08,1.670118e-08,1.670118e-08,1.670118e-08,1.670118e-08,1.670118e-08,

In [0]:
def MNB(df):
    df_dict = df.to_dict()
    new_dict = {}
    prediction = []
    #new_dict = {docIdx : {wordIdx: count},....}
    for idx in range(len(df_dict['docIdx'])):
        docIdx = df_dict['docIdx'][idx]
        wordIdx = df_dict['wordIdx'][idx]
        count = df_dict['count'][idx]
        try:
          new_dict[docIdx][wordIdx] = count
        except:
          #for unseen words in training i,e all the removed words
          new_dict[df_dict['docIdx'][idx]] = {}
          new_dict[docIdx][wordIdx] = count 
    #Calculating the scores for each doc
    for docIdx in range(1, len(new_dict)+1):
        score_dict = {}
        #Creating a probability row for each class
        for classIdx in range(1,21):
            score_dict[classIdx] = 1
            #For each word:
            for wordIdx in new_dict[docIdx]:
              try:
                probability = Pr_dict[wordIdx][classIdx]        
                power = new_dict[docIdx][wordIdx]               
                score_dict[classIdx]+=power*np.log(probability)    
              except:
                #unseen words will have score 0
                score_dict[classIdx] += 0   
            #adding prior probs         
            score_dict[classIdx] +=  np.log(prior_probs[classIdx])                          

        #Get class with max probabilty for the given docIdx 
        max_score = max(score_dict, key=score_dict.get)
        prediction.append(max_score)
        
    return prediction

In [165]:
#Get test data
test_data = open(path_to_dataset+'test.data')
df = pd.read_csv(test_data, delimiter=' ', names=['docIdx', 'wordIdx', 'count'])

#Get list of labels
test_label = pd.read_csv(path_to_dataset+'test.label', names=['t'])
test_label= test_label['t'].tolist()

#MNB Calculation
predict = MNB(df)
total = len(test_label)
val = 0
for i,j in zip(predict, test_label):
    if i == j:
        val +=1
    else:
        pass
print("Error:\t",(1-(val/total)) * 100, "%")

Error:	 20.50632911392405 %


In [168]:
#no_of_features={1072:36.9620;2067:30.766155;579:44.2638}
accuracy = {385:51.5922,872:39.1205,1868:31.8720,2866:29.1405,3865:27.5284,4863:26.369,5862:25.9826,6861:25.2498,9860:24.0506,19859:21.6389,29856:21.2924,39856:20.83944,49856:20.5063}
#118,128,132,134,135,137,138,140,140,141,144
print(accuracy)
#plt.hist(accuracy.keys(), accuracy.values(), color='g', label = "Real distribution")
#plt.show()

{385: 51.5922, 872: 39.1205, 1868: 31.872, 2866: 29.1405, 3865: 27.5284, 4863: 26.369, 5862: 25.9826, 6861: 25.2498, 9860: 24.0506, 19859: 21.6389, 29856: 21.2924, 39856: 20.83944, 49856: 20.5063}
